In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai neo4j streamlit pyngrok

DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/cyvcf2-0.30.28-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/MetaXcan-0.7.5-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/h5py_cache-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/bgen_reader-4.0.8-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
!pip install langchainhub

DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/cyvcf2-0.30.28-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/MetaXcan-0.7.5-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/h5py_cache-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /Users/JerryYaw/anaconda3/lib/python3.11/site-packages/bgen_reader-4.0.8-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [3]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [4]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_MODEL = "gpt-3.5-turbo"


In [5]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [6]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """

In [7]:
result = kg.query(cypher)
result

[{'count(n)': 2384331}]

In [8]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()
#graph.refresh_schema()
print(graph.schema)

Node properties:
Anatomy {uberon_id: STRING, anatomy_id: STRING, name: STRING, umls_cui: STRING, bto_id: STRING, mesh_id: STRING, cl_id: STRING}
Disease {disease_id: STRING, name: STRING, icd_10: STRING, umls_cui: STRING, do_id: STRING, mesh_id: STRING, icd_9: STRING, pharmgkb_id: STRING, kegg_id: STRING, iDISK_id: STRING, omim_id: STRING}
Drug {umls_cui: STRING, name: STRING, mesh_id: STRING, drugbank_id: STRING, CID: STRING, kegg_id: STRING, pharmgkb_id: STRING, drug_id: STRING, iDISK_id: STRING}
Dsp {dsp_id: STRING, iDISK_id: STRING, name: STRING}
Gene {ensembl_id: STRING, ncbi_id: STRING, symbol: STRING, hgnc_id: STRING, gene_id: STRING, pharmgkb_id: STRING}
Molecule {inchi: STRING, molecule_id: STRING, chembl_id: STRING, chebi_id: STRING, drugbank_id: STRING}
Pathway {reactome_id: STRING, pathway_id: STRING, go_id: STRING, name: STRING, kegg_id: STRING}
Sdsi {name: STRING, sdsi_id: STRING, iDISK_id: STRING}
SideEffect {name: STRING, umls_cui: STRING, side_effect_id: STRING}
Sympto

In [9]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What is BRCA1?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Gene {symbol: "BRCA1"}) RETURN g
Full Context:
[{'g': {'symbol': 'BRCA1', 'hgnc_id': '1100.0', 'pharmgkb_id': 'PA25411', 'ensembl_id': 'ENSG00000012048', 'ncbi_id': '672.0', 'gene_id': 'HGNC:1100'}}, {'g': {'symbol': 'BRCA1', 'ensembl_id': 'ENSECAG00000016974', 'ncbi_id': '100051990.0', 'gene_id': 'NCBI:100051990'}}, {'g': {'symbol': 'BRCA1', 'ensembl_id': 'ENSGALG00000002781', 'ncbi_id': '373983.0', 'gene_id': 'NCBI:373983'}}, {'g': {'symbol': 'BRCA1', 'ncbi_id': '403437.0', 'gene_id': 'NCBI:403437'}}]

> Finished chain.


{'query': 'What is BRCA1?',
 'result': 'BRCA1 is a gene with various identifiers such as symbol, hgnc_id, pharmgkb_id, ensembl_id, and ncbi_id.'}

In [11]:
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
response = chain.invoke({"query": "can you show me the pharmgkb_id of BRCA1 and its related informations?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Gene {symbol: "BRCA1"})-[:Gene_Anatomy|Gene_Pathway|Gene_Gene]->(related)
RETURN g.pharmgkb_id, related;
Full Context:
[{'g.pharmgkb_id': 'PA25411', 'related': {'name': 'FL cell', 'bto_id': 'BTO:0003130', 'anatomy_id': 'BTO:0003130'}}, {'g.pharmgkb_id': 'PA25411', 'related': {'name': 'lung cancer cell line', 'bto_id': 'BTO:0000762', 'anatomy_id': 'BTO:0000762'}}, {'g.pharmgkb_id': 'PA25411', 'related': {'name': 'Calu-6 cell', 'bto_id': 'BTO:0002235', 'anatomy_id': 'BTO:0002235'}}, {'g.pharmgkb_id': 'PA25411', 'related': {'name': 'monocytic leukemia cell line', 'bto_id': 'BTO:0002332', 'anatomy_id': 'BTO:0002332'}}, {'g.pharmgkb_id': 'PA25411', 'related': {'uberon_id': 'UBERON:0007610', 'name': 'tibial artery', 'umls_cui': 'C0085427', 'anatomy_id': 'UBERON:0007610'}}, {'g.pharmgkb_id': 'PA25411', 'related': {'name': 'T-24 cell', 'bto_id': 'BTO:0001345', 'anatomy_id': 'BTO:0001345'}}, {'g.pharmgkb_id': 'PA25411', 're

{'query': 'can you show me the pharmgkb_id of BRCA1 and its related informations?',
 'result': "I don't know the answer."}

In [12]:
chain.invoke({"query": "what are the passways in the previous query, can you list for me?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Disease)-[:Disease_Pathway]->(p:Pathway)
RETURN p.name
Full Context:
[{'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}, {'p.name': 'Glycolysis / Gluconeogenesis'}]

> Finished chain.


{'query': 'what are the passways in the previous querys, can you list for me?',
 'result': 'The pathways in the previous queries are Glycolysis / Gluconeogenesis.'}

In [13]:
chain.invoke({"query": "Can you use those passways to find associated drugs or drugid."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Pathway)<-[:Drug_Pathway]-(d:Drug)
RETURN d.drug_id
Full Context:
[{'d.drug_id': 'KEGG:D00195'}, {'d.drug_id': 'DrugBank:DB00313'}, {'d.drug_id': 'DrugBank:DB00215'}, {'d.drug_id': 'DrugBank:DB00531'}, {'d.drug_id': 'DrugBank:DB00949'}, {'d.drug_id': 'DrugBank:DB01227'}, {'d.drug_id': 'DrugBank:DB00776'}, {'d.drug_id': 'DrugBank:DB00295'}, {'d.drug_id': 'DrugBank:DB01181'}, {'d.drug_id': 'DrugBank:DB00333'}]

> Finished chain.


{'query': 'Can you use those passways to find associated drugs or drugid.',
 'result': 'Yes, you can use those pathways to find associated drugs or drug IDs.'}

In [29]:
chain.invoke({"query": "What are the drug name, and background of those drug_id"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Drug)
RETURN d.name, d.drug_id;
Full Context:
[{'d.name': 'Lepirudin', 'd.drug_id': 'DrugBank:DB00001'}, {'d.name': 'Cetuximab', 'd.drug_id': 'DrugBank:DB00002'}, {'d.name': 'Dornase alfa', 'd.drug_id': 'DrugBank:DB00003'}, {'d.name': 'Denileukin diftitox', 'd.drug_id': 'DrugBank:DB00004'}, {'d.name': 'Etanercept', 'd.drug_id': 'DrugBank:DB00005'}, {'d.name': 'Bivalirudin', 'd.drug_id': 'DrugBank:DB00006'}, {'d.name': 'Leuprolide', 'd.drug_id': 'DrugBank:DB00007'}, {'d.name': 'Peginterferon alfa-2a', 'd.drug_id': 'DrugBank:DB00008'}, {'d.name': 'Alteplase', 'd.drug_id': 'DrugBank:DB00009'}, {'d.name': 'Sermorelin', 'd.drug_id': 'DrugBank:DB00010'}]

> Finished chain.


{'query': 'What are the drug name, and background of those drug_id',
 'result': 'Lepirudin, Cetuximab, Dornase alfa, Denileukin diftitox, Etanercept, Bivalirudin, Leuprolide, Peginterferon alfa-2a, Alteplase, Sermorelin are the drug names associated with the provided drug_id.'}

In [32]:
chain.invoke({"query": "can you show me the chemical formular of those drug?"})
# The current database did not store more information from the drugbank, but we can use native chatGPT to get more useful information after this point.



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Drug)
RETURN d.name, d.chembl_id
Full Context:
[{'d.name': 'Lepirudin', 'd.chembl_id': None}, {'d.name': 'Cetuximab', 'd.chembl_id': None}, {'d.name': 'Dornase alfa', 'd.chembl_id': None}, {'d.name': 'Denileukin diftitox', 'd.chembl_id': None}, {'d.name': 'Etanercept', 'd.chembl_id': None}, {'d.name': 'Bivalirudin', 'd.chembl_id': None}, {'d.name': 'Leuprolide', 'd.chembl_id': None}, {'d.name': 'Peginterferon alfa-2a', 'd.chembl_id': None}, {'d.name': 'Alteplase', 'd.chembl_id': None}, {'d.name': 'Sermorelin', 'd.chembl_id': None}]

> Finished chain.


{'query': 'can you show me the chemical formular of those drug?',
 'result': "I don't know the chemical formula of those drugs."}

In [41]:
%%writefile app.py

import os
from dotenv import load_dotenv
import streamlit as st
from streamlit.runtime.scriptrunner.script_run_context import get_script_run_ctx

# Load environment variables from .env file
load_dotenv()


from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.graphs import Neo4jGraph


# Initialize OpenAI and Neo4j components with environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# Check if environment variables are set
if not all([OPENAI_API_KEY, OPENAI_MODEL, NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD]):
    st.error("Missing environment variables. Please set OPENAI_API_KEY, NEO4J_URI, NEO4J_USERNAME, and NEO4J_PASSWORD.")


llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model=OPENAI_MODEL,
)

embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY
)

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

import streamlit as st
from streamlit.runtime.scriptrunner.script_run_context import get_script_run_ctx

# tag::write_message[]
def write_message(role, content, save = True):
    """
    This is a helper function that saves a message to the
     session state and then writes a message to the UI
    """
    # Append to session state
    if save:
        st.session_state.messages.append({"role": role, "content": content})

    # Write to UI
    with st.chat_message(role):
        st.markdown(content)
# end::write_message[]

# tag::get_session_id[]
def get_session_id():
    return get_script_run_ctx().session_id
# end::get_session_id[]

import streamlit as st
# Create a movie chat chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a drugs expert providing associated drug information from the target gene user provided."),
        ("human", "{input}"),
    ]
)

movie_chat = chat_prompt | llm | StrOutputParser()

# Create a set of tools
from langchain.tools import Tool

tools = [
    Tool.from_function(
        name="General Chat",
        description="For general medical database chat",
        func=movie_chat.invoke,
    )
]

# Create chat history callback
from langchain_community.chat_message_histories import Neo4jChatMessageHistory

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

# Create the agent
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain import hub


from langchain_core.prompts import PromptTemplate

agent_prompt = hub.pull("hwchase17/react-chat", api_key=LANGCHAIN_API_KEY)

agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
    )

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Create a handler to call the agent
# from utils import get_session_id

def generate_response(user_input):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    response = chat_agent.invoke(
        {"input": user_input},
        {"configurable": {"session_id": get_session_id()}},)

    return response['output']

# Page Config
st.set_page_config(
    page_title="Medical Database Chatbot",
    page_icon="🧬",
    layout="wide",
    initial_sidebar_state="collapsed",
    menu_items={
        'About': 'This app provides drug information based on gene input.'
    }
)

# Set up Session State
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Hi, I'm Meda, a general medical database Chatbot!  How can I help you?"},
    ]

# Submit handler
def handle_submit(message):
    """
    Submit handler:

    context using data from Neo4j.
    """

    # Handle the response
    with st.spinner('Thinking...'):
        # Call the agent
        response = generate_response(message)
        write_message('assistant', response)


# Display messages in Session State
for message in st.session_state.messages:
    write_message(message['role'], message['content'], save=False)

# Handle any user input
if question := st.chat_input("What is up?"):
    # Display user message in chat message container
    write_message('user', question)

    # Generate a response
    handle_submit(question)


Overwriting app.py


In [42]:
import yaml
import os
from pyngrok import ngrok
import subprocess
import time

# Terminate any existing ngrok processes
ngrok.kill()

# Path to the ngrok.yml file
ngrok_config_path = os.path.expanduser("/Users/JerryYaw/Library/Application Support/ngrok/ngrok.yml")

# Load the ngrok.yml file
try:
    with open(ngrok_config_path, "r") as file:
        ngrok_config = yaml.safe_load(file)
except FileNotFoundError:
    print(f"Error: ngrok.yml file not found at {ngrok_config_path}")
    exit(1)
except yaml.YAMLError as e:
    print(f"Error reading ngrok.yml file: {e}")
    exit(1)

# Get the auth token from the configuration
auth_token = ngrok_config.get("authtoken")
if not auth_token:
    print("Error: Auth token not found in ngrok.yml file.")
    exit(1)

# Start the Streamlit app
process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])

# Optional: Wait for a short period to ensure Streamlit starts
time.sleep(5)

# Check if the Streamlit server is running
def check_server():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', 8501)) == 0

if check_server():
    # Set the auth token and establish an ngrok tunnel
    ngrok.set_auth_token(auth_token)
    public_url = ngrok.connect(8501)
    print(f"Streamlit app is live at: {public_url}")
else:
    print("Error: Streamlit server is not running on port 8501.")


2024-08-18 10:32:02.140 Port 8501 is already in use


Streamlit app is live at: NgrokTunnel: "https://6d89-70-109-199-218.ngrok-free.app" -> "http://localhost:8501"
